In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [7]:
url = "https://raw.githubusercontent.com/chatok-jnr/CSE432_Mental_Wellbeing/main/mental_health_social_media_dataset.csv"
df = pd.read_csv(url)

df.head()

,person_name,age,date,gender,platform,daily_screen_time_min,social_media_time_min,negative_interactions_count,positive_interactions_count,sleep_hours,physical_activity_min,anxiety_level,stress_level,mood_level,mental_state
0,Reyansh Ghosh,35,1/1/2024,Male,Instagram,320,160,1,2,7.4,28,2,7,6,Stressed
1,Neha Patel,24,1/12/2024,Female,Instagram,453,226,1,3,6.7,15,3,8,5,Stressed
2,Ananya Naidu,26,1/6/2024,Male,Snapchat,357,196,1,2,7.2,24,3,7,6,Stressed
3,Neha Das,66,1/17/2024,Female,Snapchat,190,105,0,1,8.0,41,2,6,6,Stressed
4,Reyansh Banerjee,31,1/28/2024,Male,Snapchat,383,211,1,2,7.1,22,3,7,6,Stressed


In [12]:
#Handle missing values, duplicates
print("\nMissing values:\n", df.isnull().sum())
print("\nDuplicates:", df.duplicated().sum())

df = df.drop_duplicates()
print("Duplicates removed.")

# Forward fill numeric & text values
df = df.ffill()

# Replace missing text with "Unknown"
for col in df.select_dtypes(include="object"):
    df[col] = df[col].fillna("Unknown")

print("Missing values handled.")





Missing values:
 person_name                    0
age                            0
date                           0
gender                         0
platform                       0
daily_screen_time_min          0
social_media_time_min          0
negative_interactions_count    0
positive_interactions_count    0
sleep_hours                    0
physical_activity_min          0
anxiety_level                  0
stress_level                   0
mood_level                     0
mental_state                   0
dtype: int64

Duplicates: 0
Duplicates removed.
Missing values handled.


In [13]:
# Clean noisy text
text_cols = ['person_name', 'gender', 'platform', 'mental_state']

for col in text_cols:
    df[col] = df[col].astype(str)
    df[col] = df[col].str.strip()
    df[col] = df[col].str.lower()
    df[col] = df[col].str.replace(r"[^a-z\s]", "", regex=True)
    df[col] = df[col].str.replace(r"\s+", " ", regex=True)

df['gender'] = df['gender'].replace({
    "m": "male",
    "man": "male",
    "malee": "male",
    "f": "female",
    "woman": "female",
    "femalee": "female",
})

df['platform'] = df['platform'].replace({
    "insta": "instagram",
    "ig": "instagram",
    "fb": "facebook",
    "snap": "snapchat",
})

df['mental_state'] = df['mental_state'].replace({
    "stress": "stressed",
    "depress": "depressed",
    "okay": "neutral"
})

print("Noisy text cleaned successfully!")
df.head()

Noisy text cleaned successfully!


,person_name,age,date,gender,platform,daily_screen_time_min,social_media_time_min,negative_interactions_count,positive_interactions_count,sleep_hours,physical_activity_min,anxiety_level,stress_level,mood_level,mental_state
0,reyansh ghosh,35,1/1/2024,male,instagram,320,160,1,2,7.4,28,2,7,6,stressed
1,neha patel,24,1/12/2024,female,instagram,453,226,1,3,6.7,15,3,8,5,stressed
2,ananya naidu,26,1/6/2024,male,snapchat,357,196,1,2,7.2,24,3,7,6,stressed
3,neha das,66,1/17/2024,female,snapchat,190,105,0,1,8.0,41,2,6,6,stressed
4,reyansh banerjee,31,1/28/2024,male,snapchat,383,211,1,2,7.1,22,3,7,6,stressed


After Drop = 


,age,gender,platform,daily_screen_time_min,social_media_time_min,negative_interactions_count,positive_interactions_count,sleep_hours,physical_activity_min,anxiety_level,stress_level,mood_level,mental_state
0,35,Male,Instagram,320,160,1,2,7.4,28,2,7,6,Stressed
1,24,Female,Instagram,453,226,1,3,6.7,15,3,8,5,Stressed
2,26,Male,Snapchat,357,196,1,2,7.2,24,3,7,6,Stressed
3,66,Female,Snapchat,190,105,0,1,8.0,41,2,6,6,Stressed
4,31,Male,Snapchat,383,211,1,2,7.1,22,3,7,6,Stressed


(5000, 13)